<h1>Criando Schemas Bronze e Silver</h1>

In [0]:
%sql
USE CATALOG medalhao;
    
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;

<h1>Trabalhando com Catalog</h1>

In [0]:
# Definição do caminho do schema Bronze
catalogo = "medalhao"
bronze_db_name = "bronze"

In [0]:
%sql
-- Limpeza da Camada Bronze
DROP TABLE IF EXISTS bronze.ft_consumidores;
DROP TABLE IF EXISTS bronze.ft_geolocalizacao;
DROP TABLE IF EXISTS bronze.ft_itens_pedidos;
DROP TABLE IF EXISTS bronze.ft_pagamentos_pedidos;
DROP TABLE IF EXISTS bronze.ft_avaliacoes_pedidos;
DROP TABLE IF EXISTS bronze.ft_pedidos;
DROP TABLE IF EXISTS bronze.ft_produtos;
DROP TABLE IF EXISTS bronze.ft_vendedores;
DROP TABLE IF EXISTS bronze.dm_categoria_produtos_traducao;
DROP TABLE IF EXISTS bronze.dm_cotacao_dolar;

In [0]:
from pyspark.sql import functions as F

def ingest_csv(nome_arquivo, nome_tabela): 
    try:
        table_name = nome_tabela
        landing_path = f"/Volumes/medalhao/default/landing/{nome_arquivo}"

        # Leitura do arquivo CSV
        df = spark.read.csv(landing_path, header=True, inferSchema=True)

        # Validação: arquivo vazio
        if df.count() == 0:
            raise ValueError(f"O arquivo {nome_arquivo} está vazio ou não pôde ser lido.")

        # Adiciona timestamp de ingestão
        df_with_metadata = df.withColumn("ingestion_timestamp", F.current_timestamp())

        # Escrita no formato Delta
        df_with_metadata.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{bronze_db_name}.{table_name}")

        print(f"✅ Tabela bronze.{nome_tabela} criada com sucesso!\n")

    except Exception as e:
        print(f"Erro ao processar {nome_tabela}: {str(e)}")

In [0]:
#dicionario de ingestão 
arquivos_para_ingestao = {
    "olist_customers_dataset.csv": "ft_consumidores",
    "olist_geolocation_dataset.csv": "ft_geolocalizacao",
    "olist_order_items_dataset.csv": "ft_itens_pedidos",
    "olist_order_payments_dataset.csv": "ft_pagamentos_pedidos",
    "olist_order_reviews_dataset.csv": "ft_avaliacoes_pedidos",
    "olist_orders_dataset.csv": "ft_pedidos",
    "olist_products_dataset.csv": "ft_produtos",
    "olist_sellers_dataset.csv": "ft_vendedores",
    "product_category_name_translation.csv": "dm_categoria_produtos_traducao",
}
for arquivo, tabela in arquivos_para_ingestao.items():
    ingest_csv(arquivo, tabela)

In [0]:
# Lista todas as tabelas do schema bronze
tabelas = [t.name for t in spark.catalog.listTables(f"{catalogo}.{bronze_db_name}")]

# Mostra 5 linhas de cada tabela
for tabela in tabelas:
    print(f"\n--- {tabela} ---")
    df = spark.table(f"{catalogo}.{bronze_db_name}.{tabela}")
    display(df.limit(5))

<h1>Ingestão da cotoção do dólar via api usando request💵</h1>

In [0]:
#solução desesperada , eu pesquisei e achei que o plano free limitava a ingestão de dados por API mas funcionou essa mesma func testava no databricks de um amigo , não sabendo o motivo deixei o codigo e injetei manualmente o resultado(esse obtido apartir da saida da mesma função)

# ingest_csv("cotacao_dolar.csv", "dm_cotacao_dolar")
# df_cotacao_bronze = spark.table(target_table)
# display(df_cotacao_bronze)

import requests
import pandas as pd
from pyspark.sql import functions as F


def ingest_cotacao_dolar(data_inicio, data_fim):

    print("\n--- INGESTÃO DA COTAÇÃO DO DÓLAR ---")

    COTACAO_TABLE_NAME = "dm_cotacao_dolar"
    
    endpoint_template = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='{data_inicio}'&@dataFinalCotacao='{data_fim}'&$select=dataHoraCotacao,cotacaoCompra&$format=json"
    api_url = endpoint_template.format(data_inicio=data_inicio, data_fim=data_fim)

    try:
        response = requests.get(api_url)
        response.raise_for_status() 
        json_data = response.json()
        cotacoes = json_data.get('value', [])
        
        if not cotacoes:
            raise ValueError("A API do Banco Central retornou uma lista vazia de cotações.")

        df_pandas = pd.DataFrame(cotacoes)
        df_cotacao_spark = spark.createDataFrame(df_pandas)
        
        df_cotacao_spark = df_cotacao_spark.withColumn("ingestion_timestamp", F.current_timestamp())

        (
            df_cotacao_spark.write.format("delta")
            .mode("overwrite")
            .saveAsTable(f"{catalogo}.{bronze_db_name}.{COTACAO_TABLE_NAME}")
        )
        print(f"✅ Tabela {catalogo}.{bronze_db_name}.{COTACAO_TABLE_NAME} criada com sucesso (API).")

    except requests.exceptions.RequestException as req_e:
        print(f"Erro de requisição à API do Banco Central: {req_e}")
    except ValueError as val_e:
        print(f"Erro de dados da API: {val_e}")
    except Exception as e:
        print(f"Erro geral ao processar a cotação: {str(e)}")

In [0]:
DATA_INICIO_FORMATADA = "01-01-2024"
DATA_FIM_FORMATADA = "03-01-2024" 


#Chamada da função para ingestão da API
ingest_cotacao_dolar(DATA_INICIO_FORMATADA, DATA_FIM_FORMATADA)